In [ ]:
# Ipython 
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# Sympy
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel

# Point mass pendulum

 Point pendulum 
```
 Coordinate system:
 O ----> y 
 |\
 | \
 |  \
 |   o (point mass)
 x   
 ```

In [ ]:
m, k, l = sp.symbols('m, k, l')
gravity = sp.symbols('g')
time = dynamicsymbols._t
theta = dynamicsymbols('theta')
thetad = dynamicsymbols('thetad')

In [ ]:
### OPTION 1 - Use a rigid body with COG displaced by l
# --- Bodies
ref = YAMSInertialBody('E') 
pend = YAMSRigidBody('P', mass=m, J_G=(0,0,0), rho_G=(l,0,0)) 
bodies=[pend] 
# --- Connections
ref.connectTo(pend, type='Joint', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(0,0,theta), rot_order='XYZ')
# --- DOFs and kinetic equations
coordinates = [theta]
speeds      = [thetad]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, bdy.mass * gravity * ref.frame.x))] # Watch out for convention here
body_loads += [(pend, (pend.frame,  -k * theta * ref.frame.z))] # restoring moment
    
# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM()
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F

In [ ]:
### OPTION 2 - - Use a rigid body with a "Free" connection that include the position and rotation
# --- Bodies
ref = YAMSInertialBody('E') 
pend = YAMSRigidBody('P', mass=m, J_G=(0,0,0), rho_G=(0,0,0)) 
bodies=[pend] 
# --- Connections
ref.connectTo(pend, type='Free', rel_pos=(l*sp.cos(theta), l*sp.sin(theta), 0), rot_type='Body', rot_amounts=(0,0,theta), rot_order='XYZ')
# --- DOFs and kinetic equations
coordinates = [theta]
speeds      = [thetad]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, bdy.mass * gravity * ref.frame.x))] # Watch out for convention here
body_loads += [(pend, (pend.frame,  -k * theta * ref.frame.z))] # restoring moment
    
# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM(simplify=True)
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F

# Rod pendulum

 Rod pendulum, of length l=2d 
```
 Coordinate system:
 O----> y 
 |\ \
 | \ \
 |  \G\    J_O = J_G + m d^2
 |   \ \
 |    \_\
 x  
 ```

In [ ]:
m, k, l = sp.symbols('m, k, d')
J_G = sp.symbols('J_G')
J_O = sp.symbols('J_O')
gravity = sp.symbols('g')
time = dynamicsymbols._t
theta = dynamicsymbols('theta')
thetad = dynamicsymbols('thetad')

In [ ]:
# --- Bodies, rigid body with COG displaced by d
ref = YAMSInertialBody('E') 
pend = YAMSRigidBody('P', mass=m, rho_G=(d,0,0), J_G =(0,0,J_G)) # NOTE: Inertia defined at COG by default. 
#pend = YAMSRigidBody('P', mass=m, rho_G=(l,0,0), J_G =(0,0,J_O), J_at_Origin=True) # NOTE: Inertia defined at origin here, not COG. 
bodies=[pend] 
# --- Connections
ref.connectTo(pend, type='Joint', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(0,0,theta), rot_order='XYZ')
# --- DOFs and kinetic equations
coordinates = [theta]
speeds      = [thetad]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, bdy.mass * gravity * ref.frame.x))] # Watch out for convention here
body_loads += [(pend, (pend.frame,  -k * theta * ref.frame.z))] # restoring moment
    
# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
EOM=model.to_EOM()
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F